# Decoding an SSP
Purpose: To demonstrate how we can use least-squares to decode the exponent of an SSP.

Let the Fourier coefs of the SSP be $P_n \in \mathbb{C}$, for $n=0, \ldots , N-1$. Also assume that $P$ is unitary, so $\| P_n \|=1$. Hence, we can write $P_n$ as $e^{i a_n}$.

When we encode $x$ in the SSP, we get $S(x) = P^x$, which means that the $n$th coef is $P_n^x = e^{i a_n x}$.

Suppose the phases, $a_n$, are sorted, such that $a_0 < a_1 < \cdots < a_{N-1}$.

The phase difference between node $j$ and $k$ can be computed:
$$
P_j \, \overline{P_k} = e^{i(a_j - a_k) x}
$$

Suppose the actual (measured) phases of the SSP are $\phi_n$. Then, the measured phase difference between nodes $j$ and $k$ is $\phi_j - \phi_k$, and we have
\begin{align}
\phi_j - \phi_k &\approx (a_j - a_k) x \\
\Delta \phi_{jk} &\approx \Delta a_{jk} x
\end{align}
If $a_j$ and $a_k$ are not too different, then there is a better chance that $\Delta \phi_{jk} \in (-\pi, \pi]$, so less chance of phase wrapping.

Suppose we formulate many such couplings,
\begin{align}
\Delta \phi_{01} &\approx \Delta a_{01} x \\
\Delta \phi_{12} &\approx \Delta a_{12} x \\
&\vdots \\
\Delta \phi_{N-2,N-1} &\approx \Delta a_{N-2,N-1} x
\end{align}

Formulating as a least-squares problem,
\begin{align}
\Delta \Phi &\approx \Delta A x \\
&\rightarrow \min_x \| \Delta \Phi - \Delta A x \|^2
\end{align}

For a chosen SSP, we know $\Delta A$, and we can get $\Delta \Phi$ from the nodes.

The only caveat is that phase-wrapping can cause outliers that disrupt the least-squares fit. So, we try to intelligently ignore outliers and solve the (robust) least-squares problem iteratively. 

This idea is taken from [Patrick Ji's MMath thesis](https://uwspace.uwaterloo.ca/handle/10012/8663).

In [1]:
from numpy import *
from matplotlib.pyplot import *
from copy import deepcopy
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:98% !important; }</style>"))
#%matplotlib notebook

## `MySP` class

In [2]:
from numpy.fft import fft, ifft
class MySP():
    def __init__(self, D=10, v=None):
        '''
         Makes a vector with Fourier coefs that are unit-modulus, and
         conjugate-symmetric.
        '''
        self.D = D
        if v is None:
            angles = random.normal(size=self.D)*2. #*pi
            #angles = -sort(-angles)
            D2 = (self.D+1)//2
            angles[-D2+1:] = flip(-angles[1:D2])
            angles[0] = 0.
            if self.D%2==0:
                angles[D2] = 0.
            self.v = real(ifft(exp(1.j*angles)))
            self.a = angles
        else:
            self.v = v
            #self._unitary()
    
    def make_zero(self):
        self.v[:] = 0.
        
    def __repr__(self):
        return str(self.v)
    
    def unitary(self):
        v = real( ifft( exp(1.j*angle(fft(self.v))) ) )
        return MySP(v=v)
    
    def __mul__(self, s):
        '''v = sp*s Scalar multiplication of SP vector'''
        return MySP(v=self.v*s)
    
    def bind(self, y):
        s = real(ifft(fft(self.v)*fft(y.v)))
        return MySP(v=s)
        
    def _bind(self, y):
        self.v = real(ifft(fft(self.v)*fft(y.v)))
        
    def unbind(self, y):
        s = real(ifft(fft(self.v)/fft(y.v)))
        return MySP(v=s)
    
    def bundle(self, y):
        s = self.v + y.v
        return MySP(v=s)
    
    def _bundle(self, y):
        self.v += y.v
        
    def power(self, p):
        v = ifft( fft(self.v)**p )
        return MySP(v=np.real(v))
    
    def similarity(self, y):
        d = np.dot(self.v, y.v)
        return d

### Demonstrate `MySP` class

In [3]:
D = 256
foo = MySP(D=D)
bar = MySP(D=D)
doh = MySP(D=D)
x = foo.bind(bar).bundle(doh).unitary()
#x = foo_bar.bundle(doh)
#foo_bar_u = foo_bar.unitary()
print(f'x = foo*bar + doh')
print(f'd(x, foo) = {x.similarity(foo)}')
print(f'd(x, bar) = {x.similarity(bar)}')
print(f'd(x, doh) = {x.similarity(doh)}')

x = foo*bar + doh
d(x, foo) = 0.15674766819389502
d(x, bar) = 0.06553053124187148
d(x, doh) = 0.6383936852814045


In [4]:
blah = x.unbind(bar)
print(f'd(x*(~bar), foo) = {blah.similarity(foo)}')
print(f'd(x*(~bar), bar) = {blah.similarity(bar)}')

d(x*(~bar), foo) = 0.6383936852814053
d(x*(~bar), bar) = 0.09556263611844382


## `HRR` class

In [5]:
class HRR():
    def __init__(self, N, D):
        self.N = N  # number of pointers in the vocabulary
        self.D = D  # dimension of vectors
        self.vocab = []
        for k in range(N):
            self.vocab.append(MySP(D=self.D))

    def __call__(self, idx):
        return self.vocab[idx]

    def cleanup(self, x):
        '''
         v, k, sim = vsa.cleanup(x)
         Return the symbol vector that is most similar to x.
         v is the vector, k is the vocab index, and sim is
         the cosine similarity.
        '''
        closest = -1
        max_sim = -1.e50
        for k,s in enumerate(self.vocab):
            sim = x.similarity(s)
            if sim>max_sim:
                closest = k
                max_sim = sim
        return self(closest), closest, max_sim

### Demonstrate `HRR` class

In [6]:
vsa = HRR(20, 256)

In [7]:
x = vsa(1).bind(vsa(2)).bundle(vsa(3)).unitary()
print(f'x = v1*v2 + v3')
print(f'd(x, v1) = {x.similarity(vsa(1))}')
print(f'd(x, v2) = {x.similarity(vsa(2))}')
print(f'd(x, v3) = {x.similarity(vsa(3))}')

x = v1*v2 + v3
d(x, v1) = 0.11293904262275037
d(x, v2) = 0.18102735615409804
d(x, v3) = 0.6232803380266955


In [8]:
blah = x.unbind(vsa(2))
print(f'd(x*(~v2), v1) = {blah.similarity(vsa(1))}')
print(f'd(x*(~v2), v2) = {blah.similarity(vsa(2))}')
print(f'd(x*(~v2), v3) = {blah.similarity(vsa(3))}')

d(x*(~v2), v1) = 0.623280338026696
d(x*(~v2), v2) = -0.013587057636206271
d(x*(~v2), v3) = 0.11293904262275029


In [9]:
# This should return 1
vsa.cleanup(blah)[1]

1

## Decoding $x$ from and SSP

In [10]:
# Create an HRR and grab one of the SSPs
vsa = HRR(20, 128)  # Number of SPs, dimension

In [11]:
sp = vsa(0)  # MySP object, index 0

In [12]:
# You can see the spatial-domain SSP
print(sp.v)

[ 0.13392781 -0.05076986 -0.079855   -0.11138068  0.05579024  0.16591792
 -0.10147318 -0.02767192  0.06870375 -0.15695813 -0.05024402 -0.02916815
 -0.06813337 -0.03895615  0.05725716  0.12523926 -0.02325628  0.02255227
 -0.0022408  -0.04539014 -0.09280159  0.07664742 -0.0191372  -0.05066483
  0.07266754 -0.00110868  0.23262652  0.12516032  0.04157838  0.00925003
  0.03069102 -0.15733725  0.00812003  0.03062124  0.03099245 -0.09589359
  0.08709031  0.07839595  0.04893242  0.05887105 -0.05359949 -0.01961888
  0.08834024 -0.05369656  0.05020649 -0.01300788 -0.07491682 -0.05079401
  0.0628185  -0.06472538  0.10148809  0.00737822  0.24784527 -0.00331239
  0.03099691  0.11865686  0.11944798  0.0624747   0.08720212  0.04536413
  0.0471169  -0.15532082 -0.01399292 -0.00434888 -0.1104933   0.10397192
 -0.17172604  0.0249859  -0.04092301  0.08072796  0.13564555 -0.0110067
 -0.02620144  0.105192    0.08622711  0.02397514  0.06891196  0.06946324
 -0.12559562  0.01868682 -0.09818786 -0.04395623  0.

In [13]:
# You can also see the original phases
print(sp.a)

[ 0.         -1.95604353 -5.45377251  1.81617672  2.08281242  0.0802225
  3.12123038 -0.14969873 -3.0723079   0.90883712 -1.21207397 -1.20233827
  2.76904074  2.62976076  1.09834027  1.72339341 -2.34483547  0.8347611
  1.62040556  3.84946244 -0.5374772  -1.65722291  0.58187879  1.4746547
 -0.11512493 -0.55184117  0.46726353  0.63977195  0.34491254  1.9782737
 -0.24433918  1.10928791 -3.27966393 -2.02974839  0.6040016  -0.5201758
 -2.01511042 -0.48839929  3.23344674 -1.93725158 -2.06665478 -2.77833146
  0.71756637  1.29576797 -1.57216695  1.19043963  1.66307984  1.62576726
  2.4503282  -0.16110547 -1.51274399  1.35099086 -1.61968437  0.08762785
 -0.73485016  0.98358565  2.99758622 -2.54998793  3.50493287 -0.18305702
 -1.16758061  0.10405475 -1.66689178  1.18774252  0.         -1.18774252
  1.66689178 -0.10405475  1.16758061  0.18305702 -3.50493287  2.54998793
 -2.99758622 -0.98358565  0.73485016 -0.08762785  1.61968437 -1.35099086
  1.51274399  0.16110547 -2.4503282  -1.62576726 -1.6630

In [14]:
# We want to create couplings by matching elements with similar phases.
# Sort the phases
sidx = argsort(sp.a)  # sidx are the indices

# Create couplings between adjacent elements...
couplings = list(zip(sidx[:-1], sidx[1:]))
# ... as well as elements one-removed (not necessary).
couplings.extend(zip(sidx[:-2], sidx[2:]))

In [15]:
# Record the Delta-A differences for the couplings
DeltaA = array([sp.a[c[0]]-sp.a[c[1]] for c in couplings])

### Now let's encode a secret $x$-value

In [16]:
true_x = random.normal()   # Shhhh... it's a secret!
spx = sp.power(true_x)     # SSP encoding true_x

In [17]:
# Get the phase differences for the couplings.
V = fft(spx.v)  # Fourier coefficients
DeltaPhi = array([angle(V[c[0]]*conj(V[c[1]])) for c in couplings])

In [18]:
x = 0.             # initial guess

kappa = 0.1        # gradient descent step size

max_iters = 100

for iter in range(1,max_iters):
    print(f'Iter {iter}: x = {x}')
    
    # Compute residual
    resid = DeltaPhi - DeltaA * x
    
    # Ignore outliers (stupid phase-wrapping!!)
    outliers = abs(resid)>1./iter
    resid[outliers] = 0.
    
    # Update x using gradient descent on least squares
    x_increment = kappa * (DeltaA @ resid)
    x += x_increment
    
    # If increments are small, stop!
    if abs(x_increment)<0.0001:
        break
    
print(f'Final estimate of x = {x}')
print(f'True value of x = {true_x}')

Iter 1: x = 0.0
Iter 2: x = 0.4835733999614269
Iter 3: x = 0.7823636892558621
Iter 4: x = 0.9669802065907144
Iter 5: x = 1.081051043877074
Iter 6: x = 1.151533122658844
Iter 7: x = 1.1950825832184162
Iter 8: x = 1.3117292974180146
Iter 9: x = 1.2351533613383407
Iter 10: x = 1.2854237356623393
Iter 11: x = 1.2524223691227834
Iter 12: x = 1.274087021676568
Iter 13: x = 1.2598646660746071
Iter 14: x = 1.2692013216413862
Iter 15: x = 1.2630720179161548
Iter 16: x = 1.2670957675219352
Iter 17: x = 1.2644542666898964
Iter 18: x = 1.2661883523883797
Iter 19: x = 1.2650499642519037
Iter 20: x = 1.2657972903790973
Iter 21: x = 1.2653066876378258
Iter 22: x = 1.2656287572684528
Iter 23: x = 1.2654173258225065
Iter 24: x = 1.2655561257964372
Final estimate of x = 1.2654650067451678
True value of x = 1.2655011180739149
